## Introduction

This notebook describes using the AWS Step Functions Data Science SDK to create and manage workflows. The Step Functions SDK is an open source library that allows data scientists to easily create and execute machine learning workflows using AWS Step Functions and Amazon SageMaker. For more information, see the following.
* [AWS Step Functions](https://aws.amazon.com/step-functions/)
* [AWS Step Functions Developer Guide](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html)
* [AWS Step Functions Data Science SDK](https://aws-step-functions-data-science-sdk.readthedocs.io)

In this notebook we will use the SDK to create steps, link them together to create a workflow, and execute the workflow in AWS Step Functions. 

In [80]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
# !{sys.executable} -m pip install -qU "stepfunctions>=2.0.0"
# !{sys.executable} -m pip show sagemaker stepfunctions

## Prequisite 

It is assumed that lambda functions for checking if model already exist or not and required IAM roles for Sagemaker, Step function is already created. <br/>
In this notebook we are going to use Step Functions SDK build-up for Sagemaker


## 1. Preprocessing logic script

Below is the preprocessing logic script which we will upload on S3 it will be used in preprocessing job. These scripts are the logic script which we have generated for preprocessing activities. Upload it on S3 and then we can use it as the parameter.

In [173]:
%%writefile titanic-score-preprocessing-xgb-script.py
# Importing required library
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
# you can put any value here according to your situation
chunksize = 10000
from sklearn import preprocessing
import glob
import time
import uuid
import datetime
runtime=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
unqstr=datetime.datetime.now().strftime("%Y%m%d%H%M%S")
path = r'/opt/ml/processing/input' # Input path
all_files = glob.glob(path + "/*.csv")
#read them into pandas
filname=all_files[0].split('/')[-1]
filname=filname.split('.')[0]
df_list = [pd.read_csv(filename,nrows=100000) for filename in all_files]
data = pd.concat(df_list)
df = data.copy()
df_list = [pd.read_csv(filename) for filename in all_files]
data = pd.concat(df_list)
df = data.copy()
#change this as per the preprocesing used for training
#create all categorical variables that we did above for both training and test sets 
df['cabin_multiple'] = df.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
df['cabin_adv'] = df.Cabin.apply(lambda x: str(x)[0])
df['numeric_ticket'] = df.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
df['ticket_letters'] = df.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
df['name_title'] = df.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
#impute nulls for continuous data 
#df.Age = df.Age.fillna(training.Age.mean())
df.Age = df.Age.fillna(df.Age.median())
#df.Fare = df.Fare.fillna(training.Fare.mean())
df.Fare = df.Fare.fillna(df.Fare.median())
#drop null 'embarked' rows. Only 2 instances of this in training and 0 in test 
df.dropna(subset=['Embarked'],inplace = True)
#tried log norm of sibsp (not used)
df['norm_sibsp'] = np.log(df.SibSp+1)
# log norm of fare (used)
df['norm_fare'] = np.log(df.Fare+1)
# converted fare to category for pd.get_dummies()
df.Pclass = df.Pclass.astype(str)
#created dummy variables from categories (also can use OneHotEncoder)
all_dummies = pd.get_dummies(df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','norm_fare','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title']])
#scaling
encoded_df = all_dummies.copy()
encoded_df['scorefilename']=df['scorefilename']
encoded_df[['Age','SibSp','Parch','norm_fare']]= scale.fit_transform(encoded_df[['Age','SibSp','Parch','norm_fare']])
encoded_df['runtime']=runtime
encoded_df['modelname']='XGboost'
encoded_df['infertype']='Batch'
encoded_df['PassengerId']=df.PassengerId.apply(lambda x:str(x)+unqstr)
encoded_df.insert(0,'runtime',encoded_df.pop('runtime'))
encoded_df.insert(1,'modelname',encoded_df.pop('modelname'))
encoded_df.insert(2,'infertype',encoded_df.pop('infertype'))
encoded_df.insert(3,'PassengerId',encoded_df.pop('PassengerId'))
encoded_df.insert(4,'scorefilename',encoded_df.pop('scorefilename'))
#encoded_df.to_csv("score_prooutput.csv",index=False,header=False)
encoded_df.to_csv("/opt/ml/processing/output/test/"+filname+"_{}.csv".format(uuid.uuid1().time_low), index=False, header=False) # test data 

Overwriting titanic-score-preprocessing-xgb-script.py


## 2. Parameter

Below are the list of paramters which we have to change inorder to run below sdk


In [150]:
import sagemaker

In [151]:
v_workflow_execution_role = "arn:aws:iam::525102048888:role/poc-sagemaker-step-functi-MachineLearningWorkflowE-1XFI2UPRXFTXE" # Step function IAM role ARN
v_preprocessing_iam_role = "arn:aws:iam::525102048888:role/service-role/AmazonSageMaker-ExecutionRole-20191105T125227" # IAM role for preprocessing container
v_lambda_execution_role = "arn:aws:iam::525102048888:role/poc-sagemaker-step-functi-LambaForDataGenerationEx-PKONGQTFWLRF"
v_preprocessing_instance_type = "ml.m5.xlarge" # Instance type for preprocessing container it changes as per workload
v_s3_input_bucket = "ds-mlops-dev" # S3 bucket for input and output data
v_prefix_for_input_data = "titanic/data/scoreinput"  # Prefix where data is stored
v_prefix_for_score_output = "titanic/data/scoreoutput"  # Prefix where data is stored
v_prefix_for_code_location = "titanic/code/titanic-score-preprocessing-xgb-script.py" # prefix where code is stored
v_score_instance_type = "ml.m5.xlarge" # Instance type for training
v_validation_scoring_instance_type = "ml.m5.large" # Instance type for batch scoring
v_model_name = "wi-mlops-titanic-ml-train-piln-xg-1350840729" # Name of DS_MLOPS model to be kept
#in above give model name to run it for XGBosst or Linear learner"
v_region = 'us-east-1' # AWS region
v_model_container = sagemaker.image_uris.retrieve('xgboost', v_region, '1.2-1') # XGboost conatiner

# VV added after design review
# sec_groups = ["sg-044e0e7ce4f5721c0"]
# subnets = ["subnet-0cf0e3f46326aa259",
#            "subnet-0156b7f5500cf0b78",
#            "subnet-032420199163cff9b"]

## 3 Import the required modules from the SDK and uploading code to s3

In [152]:
import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from sagemaker.processing import Processor,ProcessingInput, ProcessingOutput
import uuid
import sagemaker
from sagemaker.inputs import TrainingInput
import boto3
from sagemaker.network import NetworkConfig

stepfunctions.set_stream_logger(level=logging.INFO)

In [153]:
!aws s3 cp titanic-score-preprocessing-xgb-script.py s3://$v_s3_input_bucket/$v_prefix_for_code_location # Uploading preprocessing code on s3

upload: ./titanic-score-preprocessing-xgb-script.py to s3://ds-mlops-dev/titanic/code/titanic-score-preprocessing-xgb-script.py


In [170]:
!pwd

/home/ec2-user/SageMaker/WipCoe


## 4. Create workflow

In the following cell, you will define the step that you will use in our first workflow.  Then you will create, visualize and execute the workflow. 

Steps relate to states in AWS Step Functions. For more information, see [States](https://docs.aws.amazon.com/step-functions/latest/dg/concepts-states.html) in the *AWS Step Functions Developer Guide*. For more information on the AWS Step Functions Data Science SDK APIs, see: https://aws-step-functions-data-science-sdk.readthedocs.io. 

## 4.1 Creating Pre-Processing step

In [154]:
processor = Processor(image_uri='683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3',
                     role=v_preprocessing_iam_role,
                     instance_count=1,
                     instance_type=v_preprocessing_instance_type
                     #network_config = NetworkConfig(security_group_ids = sec_groups, subnets = subnets)
                     )

In [155]:
import datetime
year=datetime.datetime.now().strftime("%Y")
month=datetime.datetime.now().strftime("%m")
day=datetime.datetime.now().strftime("%d")
hour=datetime.datetime.now().strftime("%H")
print(datetime.datetime.now().strftime("%Y"))
print(datetime.datetime.now().strftime("%m"))
print(datetime.datetime.now().strftime("%d"))
print(datetime.datetime.now().strftime("%H"))
#batchscoringprefix=year,"/",month,"/",day/hour
#print(year/month/day/hour)
outputloc=("s3://{}/{}/{}/{}/{}/{}".format(v_s3_input_bucket,v_prefix_for_score_output,year,month,day,hour))
print(outputloc)

2022
08
08
08
s3://ds-mlops-dev/titanic/data/scoreoutput/2022/08/08/08


In [156]:
input_data = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_input_data)
input_code = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_code_location)
output_data = "s3://{}/{}".format(v_s3_input_bucket,"titanic/score/preprocess-data")

inputs = [
    ProcessingInput(
        source=input_data, destination="/opt/ml/processing/input", input_name="input"
    ),
    ProcessingInput(
        source=input_code,
        destination="/opt/ml/processing/input/code",
        input_name="code",
    ),
]

outputs = [
    ProcessingOutput(
        source="/opt/ml/processing/output/test",
        destination="{}/{}".format(output_data, "test"),
        output_name="test_data",
    )
    
]

In [157]:
# SageMaker expects unique names for each job, model and endpoint.
# If these names are not unique the execution will fail. Pass these dynamically for each execution using placeholders.

##VV updated after review

execution_input = ExecutionInput(
    schema={
        "PreprocessingJobName": str,
        "scoringstep":str
           }
)

In [158]:
preprocessing_step = ProcessingStep(
    state_id='Pre-processing', 
    processor=processor,
    job_name=execution_input["PreprocessingJobName"], 
    inputs=inputs, 
    outputs=outputs, 
    experiment_config=None, 
    container_entrypoint=["python3", "/opt/ml/processing/input/code/titanic-score-preprocessing-xgb-script.py"], # DS needs to change this directory /path
    wait_for_completion=True
)

## 4.5 Create a batch transform step

Now once all the above steps are done we will perform scoring on a small data set to see all the components are working fine

In [159]:
sagemaker_execution_role =  sagemaker.get_execution_role()
xgb = sagemaker.transformer.Transformer(model_name=v_model_name,
                                       instance_count=1,
                                       instance_type=v_score_instance_type,
                                    assemble_with='Line',
                                    output_path=outputloc,#"s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_score_output),
                                        accept='text/csv',
                                    base_transform_job_name='scorexgboost'
                                       )


In [160]:
from sagemaker.inputs import TransformInput

batch_scoring = TransformStep(
    state_id="batchscoring-step",
    job_name=execution_input["scoringstep"],
    transformer=xgb,
    model_name=v_model_name,
    data="{}/{}".format(output_data, "test"), # location for test data
    data_type='S3Prefix',
    content_type="text/csv",
    split_type='Line',
    wait_for_completion=True,
    input_filter="$[5:]",
    join_source='Input'
       
)

In [161]:
ses="{}/{}/".format(output_data, "test")
print(ses)

s3://ds-mlops-dev/titanic/score/preprocess-data/test/


## 4.6 Chain together steps for the basic path

The following cell links together the steps you've created into a sequential group called `basic_path`. We will chain a single step to create our basic path. See [Chain](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Chain) in the AWS Step Functions Data Science SDK documentation.

After chaining together the steps for the basic path, in this case only one step, we will visualize the basic path.

In [162]:
# First we chain the start pass state,preprocessing_step,
basic_path=Chain([preprocessing_step,batch_scoring])
#basic_path=Chain([batch_scoring])


## 4.7 Define the workflow instance

The following cell defines the [workflow](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow) with the path we just defined.

After defining the workflow, we will render the graph to see what our workflow looks like.

In [163]:
# Next, we define the workflow
basic_workflow = Workflow(
    name="titanic-xgb-score-step-function",
    definition=basic_path,
    role=v_workflow_execution_role
)

#Render the workflow
basic_workflow.render_graph()

## 4.8 Review the Amazon States Language code for your workflow

The following renders the JSON of the [Amazon States Language](https://docs.aws.amazon.com/step-functions/latest/dg/concepts-amazon-states-language.html) definition of the workflow you created. 

In [164]:
print(basic_workflow.definition.to_json(pretty=True)) # From this json we would be leveraging the codes to create the Cloud Formation parameterized template...

{
    "StartAt": "Pre-processing",
    "States": {
        "Pre-processing": {
            "Resource": "arn:aws:states:::sagemaker:createProcessingJob.sync",
            "Parameters": {
                "ProcessingJobName.$": "$$.Execution.Input['PreprocessingJobName']",
                "ProcessingInputs": [
                    {
                        "InputName": "input",
                        "AppManaged": false,
                        "S3Input": {
                            "S3Uri": "s3://ds-mlops-dev/titanic/data/scoreinput",
                            "LocalPath": "/opt/ml/processing/input",
                            "S3DataType": "S3Prefix",
                            "S3InputMode": "File",
                            "S3DataDistributionType": "FullyReplicated",
                            "S3CompressionType": "None"
                        }
                    },
                    {
                        "InputName": "code",
                        "AppManaged": fa

## 4.9 Create the workflow on AWS Step Functions

Create the workflow in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [165]:
basic_workflow.create()

[ERROR] A workflow with the same name already exists on AWS Step Functions. To update a workflow, use Workflow.update().


'arn:aws:states:us-east-1:525102048888:stateMachine:titanic-xgb-score-step-function'

In [166]:
basic_workflow.update(definition=basic_workflow.definition,role=basic_workflow.role)

[INFO] Workflow updated successfully on AWS Step Functions. All execute() calls will use the updated definition and role within a few seconds. 


'arn:aws:states:us-east-1:525102048888:stateMachine:titanic-xgb-score-step-function'

## 5 Execute the workflow

Run the workflow with [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute). Since the workflow only has a pass state, it will succeed immediately.

In [167]:
# Generate unique names for Pre-Processing Job, Training Job, and Model Evaluation Job for the Step Functions Workflow
 # Each Training Job requires a unique name
preprocessing_job_name = "titanic-xgb-score-prep-{}".format(
    uuid.uuid1().hex
)  # Each Preprocessing job requires a unique name,
scoring_job_name = "titanic-xgb-score-{}".format(
    uuid.uuid1().hex
)  # Each Evaluation Job requires a unique name


In [168]:
basic_workflow_execution = basic_workflow.execute(
    inputs={
       "PreprocessingJobName": preprocessing_job_name,
        "scoringstep":scoring_job_name  # Each pre processing job (SageMaker processing job) requires a unique name,
            }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


## 5.1 Review the execution progress

Render workflow progress with the [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress).

This generates a snapshot of the current state of your workflow as it executes. This is a static image. Run the cell again to check progress. 

In [169]:
basic_workflow_execution.render_progress()